In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
    sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [5]:
def value_iteration(env, theta = 0.0001, discount_factor = 1.0):
    """
    Value Iteration Algorithm
    
    Args:
        env: OpenAI environment. env.P represents the transition probabilities of the environment.
        theta: Stopping threshold. If the value of all states changes less than theta
            in one iteration we are done.
        discount_factor: lambda time discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    
    while True:
        V_update = np.zeros(env.nS)
        for s in range(env.nS):
            v = np.zeros(env.nA)
            for a in range(env.nA):
                for (prob, next_state, reward, done) in env.P[s][a]:
                    v[a] += prob*reward + prob*V[next_state]
            V_update[s] = np.max(v)
        if np.max(np.abs(V_update - V)) < theta:
            break
        else:
            V = V_update 

    q_value = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        for a in range(env.nA):
            reward = 0
            v = 0
            for (prob, next_state, reward, done) in env.P[s][a]:
                reward += prob*reward
                v += prob*V[next_state]
            q_value[s,a] = reward + v
            
    selected_action = np.argmax(q_value, axis = 1).tolist()
    for row, i in enumerate(selected_action):
        policy[row, i] = 1
    return policy, V

In [6]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [7]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)